In [1]:
from utils import load_model_and_files, get_model_details, reconstruct_word, merge_results, preprocess_text, preprocess_stopwords, preprocess_lemmatization

### carico i modelli

In [3]:
# 1. Recupera gli ID dei modelli e i relativi file dal file .env
(model_1_id, model_1_files), (model_2_id, model_2_files) = get_model_details()

# 2. Carica le pipeline per i modelli
print(model_1_id)
model_1_pipeline = load_model_and_files(model_1_id, model_1_files)

print(model_2_id)
model_2_pipeline = load_model_and_files(model_2_id, model_2_files)

osiria/bert-italian-uncased-ner


Device set to use cpu


IVN-RIN/MedPsyNIT


Device set to use cpu


### faccio una prova utilizzando direttamente i modelli sul testo crudo

In [4]:
# 3. Definisci il testo su cui eseguire l'analisi
text = "Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie"

# 4. Esegui l'analisi sui modelli solo se le pipeline sono caricate correttamente
result_1 = []
if model_1_pipeline:
    result_1 = model_1_pipeline(text)
else:
    print(f"Errore nel caricare la pipeline per il modello 1: {model_1_id}")

result_2 = []
if model_2_pipeline:
    result_2 = model_2_pipeline(text)
else:
    print(f"Errore nel caricare la pipeline per il modello 2: {model_2_id}")

# 5. Ricostruisci i risultati dai sub-token in parole complete per entrambi i modelli
reconstructed_model_1_results = reconstruct_word(result_1) if result_1 else []
reconstructed_model_2_results = reconstruct_word(result_2) if result_2 else []

# 6. Unisci i risultati dei due modelli, con la preferenza per il modello 2
final_results = merge_results(reconstructed_model_1_results, reconstructed_model_2_results)

# 7. Stampa i risultati finali in modo leggibile
print(text)
print("\nRisultati finali dopo unione dei modelli:")
for result in final_results:
    print(f"Parola: {result['word']}, Entità: {result['entity']}, Score: {result['score']:.4f}")
    print("--------------------------------")

Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie

Risultati finali dopo unione dei modelli:
Parola: giulia, Entità: PER, Score: 0.9279
--------------------------------
Parola: milano, Entità: LOC, Score: 0.7912
--------------------------------
Parola: endometriosi, Entità: DIAGNOSI E COMORBIDITA (B), Score: 0.7861
--------------------------------
Parola: fibromi, Entità: DIAGNOSI E COMORBIDITA (B), Score: 0.8488
--------------------------------
Parola: uterino, Entità: DIAGNOSI E COMORBIDITA (I), Score: 0.7146
--------------------------------


### faccio una prova puleno il testo

In [7]:
# 3. Definisci il testo su cui eseguire l'analisi
text = "Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie"

text_processed = preprocess_text(text)
text_processed = preprocess_stopwords(text_processed)
text_processed = preprocess_lemmatization(text_processed)

# 4. Esegui l'analisi sui modelli solo se le pipeline sono caricate correttamente
result_1 = []
if model_1_pipeline:
    result_1 = model_1_pipeline(text_processed)
else:
    print(f"Errore nel caricare la pipeline per il modello 1: {model_1_id}")

result_2 = []
if model_2_pipeline:
    result_2 = model_2_pipeline(text_processed)
else:
    print(f"Errore nel caricare la pipeline per il modello 2: {model_2_id}")

# 5. Ricostruisci i risultati dai sub-token in parole complete per entrambi i modelli
reconstructed_model_1_results = reconstruct_word(result_1) if result_1 else []
reconstructed_model_2_results = reconstruct_word(result_2) if result_2 else []

# 6. Unisci i risultati dei due modelli, con la preferenza per il modello 2
final_results = merge_results(reconstructed_model_1_results, reconstructed_model_2_results)

# 7. Stampa i risultati finali in modo leggibile
print(text)
print(text_processed)
print("\nRisultati finali dopo unione dei modelli:")
for result in final_results:
    print(f"Parola: {result['word']}, Entità: {result['entity']}, Score: {result['score']:.4f}")
    print("--------------------------------")

Vorrei sapere se Giulia che vive Milano c'è qualche (seria)speranza anche per chi - dopo anni di problemi seri (endometriosi e fibromi uterino) - sta raggiungendo i 50 anni. (Sono consapevole di essere anzianotta, ma la speranza c'è sempre). Grazie
volere sapere Giulia vivere Milano c ' serio speranza problema serio endometrioso fibrome uterino raggiungere il 50 . consapevole anzianotta , speranza c ' .

Risultati finali dopo unione dei modelli:
Parola: giulia, Entità: MISC, Score: 0.3940
--------------------------------
Parola: milano, Entità: MISC, Score: 0.5187
--------------------------------
Parola: speranza, Entità: MISC, Score: 0.3922
--------------------------------
Parola: endometrioso, Entità: DIAGNOSI E COMORBIDITA (B), Score: 0.4980
--------------------------------
Parola: fime, Entità: DIAGNOSI E COMORBIDITA (I), Score: 0.5342
--------------------------------
Parola: uterino, Entità: DIAGNOSI E COMORBIDITA (I), Score: 0.6979
--------------------------------
